### 衡量预测结果的 BDBR 性能 -> VMAF

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 6)

import seaborn as sns
import matplotlib.pyplot as plt

plt.rcParams.update({
    'font.family': 'serif',  # 使用衬线字体
    'font.serif': ['Times New Roman'],  # 指定 Times New Roman 字体
    'font.size': 11,
    'text.usetex': False,
    'axes.labelsize': 11,
    'axes.titlesize': 11,
    'xtick.labelsize': 11,
    'ytick.labelsize': 11,
    'legend.fontsize': 9,
    'legend.title_fontsize': 9,
    'savefig.bbox': 'tight',
    'lines.linewidth': 2,
    'lines.markersize': 6,
    'grid.linewidth': 1,
    'axes.linewidth': 1
})
sns.set_style("whitegrid")
flierprops = dict(marker='o', markersize=2, linestyle='none')

table_dir = "/home/zhaoy/asset-fastCAE/results/vvenc/tables"
fig_dir   = "/home/zhaoy/asset-fastCAE/results/vvenc/figs"

In [2]:
def rm_outliers(df, cols, threshold=1.5, lower_only=True):
    for col in cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - (threshold * IQR)
        upper_bound = Q3 + (threshold * IQR)

        if lower_only:
            df = df[df[col] >= lower_bound]
        else:
            df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

    return df

In [3]:
import os
import gc
import numpy as np
from scipy.spatial import ConvexHull

cols = ["seqName", "sceneId", "regressor", "func", "input", "preset", "size", "qp", "pred_target"]
def process_group(args):
    seqName, sceneId, preset, group, regressor, inputs, func, distortion = args
    
    distortion = distortion.lower()
    assert distortion.lower() in ["psnr", "log2psnr", "ssim", "log2ssim", "vmaf", "log2vmaf"]
    is_log_distortion = ("log2" in distortion.lower())
    distortion_type   = distortion.split("log2")[-1]
     
    bitrate_df = group[
        (group["target"] == "log2bitrate") & 
        (group["regressor"] == regressor) & 
        (group["func"] == "quadratic2") &       # log2bitrate 固定使用 quadratic2
        (group["input"] == inputs)
    ][cols + ["log2bitrate", "bitrate"]].reset_index(drop=True)
    
    distortion_df = group[
        (group["target"] == distortion) & 
        (group["regressor"] == regressor) & 
        (group["func"] == func) & 
        (group["input"] == inputs)
    ][cols + [f"log2{distortion_type}", distortion_type]].reset_index(drop=True)
    
    bitrate_df    = bitrate_df.rename(columns={"pred_target": "pred_log2bitrate"})
    distortion_df = distortion_df.rename(columns={"pred_target": f"pred_{distortion}"})
    
    # 扔掉 bitrate_df 的 func -> 避免干扰 merge
    bitrate_df = bitrate_df.drop(columns=["func"])
    
    group_rd = pd.merge(distortion_df, bitrate_df, how="inner")
    group_rd["pred_bitrate"] = 2 ** group_rd["pred_log2bitrate"]
    if is_log_distortion:
        group_rd[f"pred_{distortion_type}"] = 2 ** group_rd[f"pred_{distortion}"]
    
    inputs = inputs.replace("(", "").replace(")", "")
    
    """ 1. 得到凸包 """
    points = group_rd[["bitrate", f"{distortion_type}"]].values
    actual_convex = ConvexHull(points)
    actual_hull_points = points[actual_convex.vertices]
    actual_hull_points = actual_hull_points[np.argsort(actual_hull_points[:, 0])]
    
    points = group_rd[["pred_bitrate", f"pred_{distortion_type}"]].values
    pred_convex  = ConvexHull(points)
    pred_hull_points = points[pred_convex.vertices]
    pred_hull_points = pred_hull_points[np.argsort(pred_hull_points[:, 0])]
    
    # 保存凸包所在各行
    pred_convex_df = group_rd.iloc[pred_convex.vertices].reset_index(drop=True)
    actual_convex_df = group_rd.iloc[actual_convex.vertices].reset_index(drop=True)
    pred_convex_df["convex"] = "pred"    
    actual_convex_df["convex"] = "actual"
    
    # 保存 fixed QP 对应各行 (QP = 27)
    fixed_convex_df = group_rd[group_rd["qp"] == 27].reset_index(drop=True)
    fixed_convex_df["convex"] = "fixed"
    
    os.makedirs(f"{table_dir}/BDBR/rd-{distortion}/{seqName}-scene{sceneId}", exist_ok=True)
    
    convex_df = pd.concat([pred_convex_df, actual_convex_df], axis=0)    
    convex_df = pd.concat([convex_df, fixed_convex_df], axis=0).reset_index(drop=True)
    convex_df.to_csv(f"{table_dir}/BDBR/rd-{distortion}/{seqName}-scene{sceneId}/{seqName}-scene{sceneId}_{regressor}_{func}_{preset}_{inputs}.csv", index=False)
    
    """ 2. 画图 """
    fig, axes = plt.subplots(1, 2, figsize=(9, 4), sharey=True)
    sns.lineplot(ax=axes[0], data=group_rd, x="pred_bitrate", y=f"pred_{distortion_type}", hue="size", marker="o")            
    axes[0].plot(pred_hull_points[:, 0], pred_hull_points[:, 1], linestyle='--', color='purple', marker="x", lw=2, alpha=0.6, label="predicted convex hull")
    axes[0].legend()
    axes[0].set_xlabel("Predicted Bitrate (kbps)")
    ylabel = "PSNR (dB)" if distortion_type == "psnr" else distortion_type.upper()
    axes[0].set_ylabel(f"Predicted {ylabel}")
    
    sns.lineplot(ax=axes[1], data=group_rd, x="bitrate", y=distortion_type, hue="size", marker="o")
    axes[1].plot(actual_hull_points[:, 0], actual_hull_points[:, 1], linestyle='--', color='purple', marker="x", lw=2, alpha=0.6, label="actual convex hull")
    axes[1].legend()
    axes[1].set_xlabel("Actual Bitrate (kbps)")
    axes[1].set_ylabel(f"Actual {ylabel}")
    
    x_min = min(axes[0].get_xlim()[0], axes[1].get_xlim()[0])
    x_max = max(axes[0].get_xlim()[1], axes[1].get_xlim()[1])
    y_min = min(axes[0].get_ylim()[0], axes[1].get_ylim()[0])
    y_max = max(axes[0].get_ylim()[1], axes[1].get_ylim()[1])
    
    axes[0].set_xlim(x_min, x_max)
    axes[1].set_xlim(x_min, x_max)
    axes[0].set_ylim(y_min, y_max)
    axes[1].set_ylim(y_min, y_max)
    
    plt.subplots_adjust(wspace=0.1)
    plt.tight_layout()
    
    os.makedirs(f"{fig_dir}/BDBR/rd-{distortion}/{seqName}-scene{sceneId}", exist_ok=True)
    plt.savefig(f"{fig_dir}/BDBR/rd-{distortion}/{seqName}-scene{sceneId}/{seqName}-scene{sceneId}_{regressor}_{func}_{preset}_{inputs}.pdf", format="pdf")
    plt.close("all")
    gc.collect()

#### 1. 预测的是 log2vmaf

In [4]:
import pandas as pd

pred_df = pd.read_csv(f"{table_dir}/predCurve/combined_preds.csv")
pred_df = pred_df.drop(columns=["p1", "p2", "p3", "pred_p1", "pred_p2", "pred_p3"])     # 不用看参数了

display(pred_df)
print(pred_df.columns.tolist())

,seqName,sceneId,regressor,input,preset,size,func,target,qp,pred_target,bitrate,log2bitrate,psnr,log2psnr,ssim,log2ssim,vmaf,log2vmaf,err
0,Lecture-42c3,0,Adam,(faster)x(360P),faster,1080P,linear,bitrate,22,675.249878,1038.4604,10.020230,54.06102,5.756517,0.998422,-0.002279,97.265945,6.603863,363.210522
1,Lecture-42c3,0,Adam,(medium)x(360P),faster,1080P,linear,bitrate,22,695.509933,1038.4604,10.020230,54.06102,5.756517,0.998422,-0.002279,97.265945,6.603863,342.950467
2,Lecture-42c3,0,Adam,(slower)x(360P),faster,1080P,linear,bitrate,22,1486.697418,1038.4604,10.020230,54.06102,5.756517,0.998422,-0.002279,97.265945,6.603863,-448.237018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4257789,Lecture-7e40,1,SGDRegressor,(faster)x(1080P),slower,720P,quadratic3,log2vmaf,52,6.092571,57.5278,5.846187,32.51932,5.023225,0.944059,-0.083052,73.387421,6.197461,0.104890
4257790,Lecture-7e40,1,SGDRegressor,(medium)x(1080P),slower,720P,quadratic3,log2vmaf,52,6.029500,57.5278,5.846187,32.51932,5.023225,0.944059,-0.083052,73.387421,6.197461,0.167961
4257791,Lecture-7e40,1,SGDRegressor,(slower)x(1080P),slower,720P,quadratic3,log2vmaf,52,5.804019,57.5278,5.846187,32.51932,5.023225,0.944059,-0.083052,73.387421,6.197461,0.393442


['seqName', 'sceneId', 'regressor', 'input', 'preset', 'size', 'func', 'target', 'qp', 'pred_target', 'bitrate', 'log2bitrate', 'psnr', 'log2psnr', 'ssim', 'log2ssim', 'vmaf', 'log2vmaf', 'err']


In [ ]:
""" 计算所有序列的 RD 性能 """
grouped = pred_df.groupby(["seqName", "sceneId", "preset"], as_index=False)

from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor

tasks = []
verbose = True
for (seqName, sceneId, preset), group in tqdm(grouped):
    for regressor in group["regressor"].unique():
        for func in group["func"].unique():
            for inputs in group["input"].unique():
                tasks.append((seqName, sceneId, preset, group, regressor, inputs, func, "log2vmaf"))
                verbose = False

with ProcessPoolExecutor() as executor:
    executor.map(process_group, tasks)

100%|██████████| 66/66 [00:05<00:00, 11.48it/s]


In [ ]:
""" 把每个序列所有的 dataframe 叠在一起 """
import pandas as pd
from tqdm import tqdm

seqs = os.listdir(f"{table_dir}/BDBR/rd-log2vmaf")
convex_df = pd.DataFrame()

for seq in seqs:
    for f in tqdm(os.listdir(f"{table_dir}/BDBR/rd-log2vmaf/{seq}")):
        convex_df = pd.concat([convex_df, pd.read_csv(f"{table_dir}/BDBR/rd-log2vmaf/{seq}/{f}")], axis=0)

convex_df = convex_df.reset_index(drop=True) 
display(convex_df)

convex_df.to_csv(f"{table_dir}/BDBR/convex_log2vmaf.csv", index=False)

In [ ]:
""" 计算 BDBR -> 1) 相较于传统 CAE (会有 loss);  2) 相较于 fixed QP (会有 gain) """
from fastCAE.metrics.bjontegaard import BD_PSNR, BD_RATE
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 8)

grouped = convex_df.groupby(["seqName", "sceneId", "regressor", "func", "input", "preset"])
rows = []
for i, ((seqName, sceneId, regressor, func, inputs, preset), group) in tqdm(enumerate(grouped)):
    pred_convex_df   = group[group["convex"] == "pred"].sort_values(by="bitrate").reset_index(drop=True)
    actual_convex_df = group[group["convex"] == "actual"].sort_values(by="bitrate").reset_index(drop=True)
    fixed_convex_df  = group[group["convex"] == "fixed"].sort_values(by="bitrate").reset_index(drop=True)
    
    bd_vmaf_cae = BD_PSNR(
        actual_convex_df["bitrate"].tolist(), actual_convex_df["vmaf"].tolist(), 
        pred_convex_df["bitrate"].tolist(), pred_convex_df["vmaf"].tolist(), piecewise=0
    )
    bd_vmaf_fix = BD_PSNR(
        fixed_convex_df["bitrate"].tolist(), fixed_convex_df["vmaf"].tolist(), 
        pred_convex_df["bitrate"].tolist(), pred_convex_df["vmaf"].tolist(), piecewise=0
    )
    rows.append([seqName, sceneId, regressor, func, inputs, preset, bd_vmaf_cae, "cae"])
    rows.append([seqName, sceneId, regressor, func, inputs, preset, bd_vmaf_fix, "fixed"])
    
all_bdbr = pd.DataFrame(rows, columns=["seqName", "sceneId", "regressor", "func", "input", "preset", "bd_vmaf", "anchor"])
all_bdbr = rm_outliers(all_bdbr, cols=["bd_vmaf"], threshold=0.75)
display(all_bdbr)
display(all_bdbr.describe())                  

all_bdbr.to_csv(f"{table_dir}/BDBR/all_rd_log2vmaf.csv", index=False)

In [ ]:
# 看一下不同预测策略 (regressor, func, input) 的效果
table_dir = "/home/zhaoy/asset-fastCAE/results/vvenc/tables"
all_bdbr = pd.read_csv(f"{table_dir}/BDBR/all_rd_log2vmaf.csv")

grouped = all_bdbr.groupby(["anchor", "regressor", "func", "input"], as_index=False)

rows = []
for (anchor, regressor, func, inputs), group in grouped:
    group = group.reset_index(drop=True)
    rows.append([regressor, func, inputs, group["bd_vmaf"].mean(), anchor])

avg_bdbr = pd.DataFrame(rows, columns=["regressor", "func", "input", "bd_vmaf", "anchor"])
avg_bdbr = rm_outliers(avg_bdbr, cols=["bd_vmaf"], threshold=0.75).reset_index(drop=True)
display(avg_bdbr)

avg_bdbr.to_csv(f"{table_dir}/BDBR/avg_rd_log2vmaf.csv", index=False)

In [ ]:
# 直观看一下区别
label_mapping = {
    'Adam': 'Adam',
    'LinearRegression': 'Linear',
    'LinearSVR': 'SVR',
    'RMSProp': 'RMSProp',
    'RandomForestRegressor': 'RF',
    'SGDRegressor': 'SGD'
}

# 画一下 BD-Rate (左边是相比 fixed QP 的, 右边是相比 traditional CAE 的)
fig, axes = plt.subplots(2, 1, figsize=(5, 6), sharex=True)

sns.boxplot(
    ax=axes[0], data=avg_bdbr[(avg_bdbr["bd_vmaf"] < 0.08) & (avg_bdbr["bd_vmaf"] > 0) & (avg_bdbr["anchor"] == "fixed") & (avg_bdbr["regressor"] != "Adam") & (avg_bdbr["regressor"] != "RMSProp")], x="func", y="bd_vmaf", hue="regressor", flierprops=flierprops)
handles, labels = axes[0].get_legend_handles_labels()
new_labels = [label_mapping[label] for label in labels]
axes[0].set_xlabel("")
axes[0].set_ylabel("BD-VMAF vs. fixed bitrate ladder")
axes[0].legend(handles, new_labels, borderaxespad=0., loc="lower right")

sns.boxplot(ax=axes[1], data=avg_bdbr[(avg_bdbr["bd_vmaf"] < 0) & (avg_bdbr["anchor"] == "cae") & (avg_bdbr["regressor"] != "Adam") & (avg_bdbr["regressor"] != "RMSProp")], x="func", y="bd_vmaf", hue="regressor", flierprops=flierprops)
handles, labels = axes[1].get_legend_handles_labels()
new_labels = [label_mapping[label] for label in labels]
axes[1].set_xlabel("")
axes[1].set_ylabel("BD-VMAF vs. complete pre-encoding")
axes[1].legend(handles, new_labels, borderaxespad=0., loc="lower right")

fig_dir = "/home/zhaoy/asset-fastCAE/results/vvenc/figs"
plt.tight_layout()
plt.savefig(f"{fig_dir}/BDBR/bd_vmaf_log2vmaf.pdf", format="pdf")